In [7]:
import pandas as pd
xsls_doc_name="/Users/amit/Documents/IFC/gil_school.xlsx"

In [190]:
#pd.read_csv(doc_name)
df=pd.read_excel(xsls_doc_name).dropna(axis=1, how='all').dropna(axis=0, how='all')

In [191]:
import re
def get_compiled_pattern(pattern):
    return(re.compile(pattern))
    
compiled_section=get_compiled_pattern("^\d+$")
compiled_sub_section=get_compiled_pattern("^\d+.\d+$")
compiled_task=get_compiled_pattern("^\d+\.\d+\.\d+$")
compiled_item=get_compiled_pattern("^\d+\.\d+\.\d+\.\d+$")
    

In [215]:
import numpy as np
df["item_id"]=df["סעיף"].astype("str")
df["description"]=df["תאור"].astype("str")
df["row_type"]=df.apply(lambda x: "section" if compiled_section.match(x.item_id) else None, axis=1)
df["row_type"]=df.apply(lambda x: "sub_section" if compiled_sub_section.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "task" if compiled_task.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "item" if compiled_item.match(x.item_id) else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "total_cost" if  "סה''כ" in x.description else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "redundant_title" if  "תאור" == x.description else x.row_type, axis=1)
df["row_type"]=df.apply(lambda x: "comment" if x.row_type==None and x.description!="nan" else x.row_type, axis=1)


In [219]:
projects={'gil_school'}
section_list=[]
current_section=None
tc_1st =True
for index, row in df.iterrows():
    if (row['description']=='nan'):
        continue
    if(row['row_type']=="section"):
        if (current_section!=None):
            section_list.append(current_section)
        current_section={"section_name": row['description'], "item_id": row['item_id'], "sub_sections_list":[]}
    if(row['row_type']=="sub_section"):
        sub_section={"sub_section_name": row['description'], "item_id": row['item_id'], "tasks_list":[]}
        current_section['sub_sections_list'].append(sub_section)
    if(row['row_type']=="task"):
        current_task={"task_name": row["description"] ,"item_id": row['item_id'], "items_list":[] }
        current_section['sub_sections_list'][-1]['tasks_list'].append(current_task)
    if(row['row_type']=="item"):
        current_item={"item_name": row["description"] ,"item_id": row['item_id'], "units":row["יח'"], "quantity": row["כמות"], "price": row["מחיר"], "item_cost": row['סה"כ']}
        current_section['sub_sections_list'][-1]['tasks_list'][-1]["items_list"].append(current_item)
    if(row['row_type']=="total_cost"):
        if tc_1st:
            current_section['sub_sections_list'][-1]['tasks_list'][-1]["task_cost"]=row['סה"כ']
            tc_1st=False
        else:
            current_section['sub_sections_list'][-1]["sub_section_cost"]=row['סה"כ']
            tc_1st=True
    if(row['row_type']=="comment"):
        current_section['sub_sections_list'][-1]["comment"]=row["description"]
        
#    print(row['row_type'], row['description'])
section_list
project={'sections': section_list}

In [221]:
import json 
project
with open("/Users/amit/Documents/IFC/gil_school.json", "w") as outfile: 
    json.dump(project, outfile)

In [2]:
#import pdfplumber
#from pdfreader import SimplePDFViewer
from pypdf import PdfReader
#def extract_text_with_pdfplumber(pdf_path):
#    text = ""
#    with pdfplumber.open(pdf_path) as pdf:
#        for page in pdf.pages:
#            text += page.extract_text()
#    return text

from transformers import MarianMTModel, MarianTokenizer

def translate_hebrew_to_english(text):
    # Define the model name for Hebrew to English translation
    model_name = 'Helsinki-NLP/opus-mt-mul-en'
    
    # Load the pre-trained model and tokenizer
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    
    # Generate the translated text
    translated = model.generate(**inputs)
    
    # Decode the translated text
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    
    return translated_text

def extract_text_with_pdfReader(pdf_path):
    text = ""
#    english_text=""
    english_text = ""
    pdf = PdfReader(pdf_path)
    for page in pdf.pages:
        text += page.extract_text()
#        english_text += translate_hebrew_to_english(text)
    return text
path="/Users/amit/Documents/IFC/gil_school.pdf"
he_text = extract_text_with_pdfReader(path)


In [29]:
import re
child_pattern = r"\d{2}\.\d{2}\.\d{3}[^\d.]"
parent_pattern=r"\n\d{2}\.\d{2}\w"
matches = re.findall(pattern, he_text)


In [30]:
#print(f"Number of occurrences: {len(matches)}")
#print("Matches:", matches)
parent_documents = re.split(parent_pattern, he_text)

In [36]:
file_path = "/Users/amit/github/rag_neo4j_streamlit/neo4j_rags/packages/neo4j-advanced-rag/PdfReader_example.txt"
with open(file_path, 'r', encoding='utf-8') as file:
        text=file.read()

In [35]:

for i, parent in enumerate(parent_documents):
#    print(f"parent chank {i + 1} XXXXXXXXXXXXXXXXXXXXXXXXXXX:")
#    child_documents = re.split(child_pattern, parent)
#    for j, child in enumerate(child_documents):
#        print(f"child chank {j + 1} YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
#        print(child_documents)
#        print()

In [4]:
def write_string_to_file(file_path, content):
    with open(file_path, 'w') as file:
        file.write(content)

# Example usage
file_path = '/Users/amit/github/rag_neo4j_streamlit/neo4j_rags/packages/neo4j-advanced-rag/PdfReader_example.txt'
write_string_to_file(file_path, he_text)

In [ ]:
#!pip install transformers torch
from transformers import MarianMTModel, MarianTokenizer

# Load model and tokenizer for English to French translation
model_name = 'Helsinki-NLP/opus-mt-mul-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


In [ ]:
en_text=""
for s in he_text.split("\n"):
    print(s)
    en_text += translate_hebrew_to_english (he_text)
    

In [ ]:
def write_string_to_file(file_path, content):
    with open(file_path, 'w') as file:
        file.write(content)

# Example usage
file_path = '/Users/amit/github/rag_neo4j_streamlit/neo4j_rags/packages/neo4j-advanced-rag/PdfReader_example.txt'
write_string_to_file(file_path, pdf_text)

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

def translate_hebrew_to_english(text):
    # Define the model name for Hebrew to English translation
    model_name = 'Helsinki-NLP/opus-mt-mul-en'
    
    # Load the pre-trained model and tokenizer
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    
    # Generate the translated text
    translated = model.generate(**inputs)
    
    # Decode the translated text
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    
    return translated_text


In [ ]:
pdf_text[:500]

In [ ]:
translate_hebrew_to_english(pdf_text)

In [ ]:
en_text

In [ ]:
with open("/Users/amit/github/rag_neo4j_streamlit/neo4j_rags/packages/neo4j-advanced-rag/apt_mecher.txt", "w") as file:
    # Write the string to the file
    file.write(pdf_text)

In [ ]:
# importing required classes 
from pypdf import PdfReader 
  
# creating a pdf reader object 
#reader = PdfReader('/Users/amit/private/amit_cahanovich.pdf') 
reader=PdfReader('/Users/amit/Documents/IFC/ronen_aptmt.pdf') 
  
# printing number of pages in pdf file 
print(len(reader.pages)) 
  
# creating a page object 
page = reader.pages[0] 
  
# extracting text from page 
print(page.extract_text()) 



In [ ]:
for page in reader.pages:
    print(page.extract_text())